In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %%bash
# set -m
# git clone https://github.com/sachdevkartik/DeepLense.git
# cd DeepLense && git checkout kartik_contribution
# cd ..
# mv DeepLense/Transformers_Classification_DeepLense_Kartik_Sachdev/* .
# rm -rf DeepLense

In [3]:
# %%bash
# pwd

In [4]:
!nvidia-smi

Sun Jul 30 15:35:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    31W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# %%bash
# pip3 install --upgrade -r requirements.txt

In [6]:
# !pip uninstall numpy

In [7]:
# !pip install numpy

In [8]:
from __future__ import print_function
import os
import time
import copy
import json
import yaml

import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
import torchvision
from typing import *

In [9]:
from utils.util import *
from utils.inference import InferenceSSL
from argparse import ArgumentParser
from config.data_config import DATASET
from torch.utils.data import DataLoader, random_split
from models.cnn_zoo import CustomResNet
from models.byol import BYOL, BYOLSingleChannel


from utils.dataset import DefaultDatasetSetupSSL
from self_supervised.losses.contrastive_loss import (
    ContrastiveLossEuclidean,
    ContrastiveLossEmbedding,
    SimCLR_Loss,
    NegativeCosineSimilarity,
)
from utils.train import (
    train_simplistic,
    train_byol,
    train_contrastive_pair,
    train_contrastive_with_labels,
    train_contrastive,
)
from self_supervised.losses.sym_neg_cos_sim_loss import SymNegCosineSimilarityLoss

from models.modules.head import BYOLProjectionHead, BYOLPredictionHead

In [10]:
args = {
    "dataset_name": "Model_II",
    "save": "/content/drive/MyDrive/deeplense/data",
    "num_workers": 8,
    "train_config_path": "self_supervised/config/resnet_byol.yaml",
    "cuda": True,
    "log_dir": "logger"
}

In [11]:
dataset_name = args["dataset_name"]
dataset_dir = args["save"]
use_cuda = args["cuda"]
num_workers = args["num_workers"]
train_config_path = args["train_config_path"]
log_dir_base = args["log_dir"]

In [12]:
classes = DATASET[f"{dataset_name}"]["classes"]
num_classes = len(classes)

# Open the YAML file and load its contents
with open(train_config_path, "r") as file:
    train_config = yaml.safe_load(file)

In [13]:
batch_size = 64
epochs_pretrained = train_config["pretrained"]["num_epochs"]
epochs_finetuned = train_config["finetuned"]["num_epochs"]

learning_rate = train_config["optimizer_config"]["lr"]
margin = train_config["ssl"]["margin"]
num_channels = train_config["channels"]
temperature = train_config["ssl"]["temperature"]
network_type = train_config["network_type"]
image_size = train_config["image_size"]
optimizer_config = train_config["optimizer_config"]

backbone = train_config["backbone"]

make_directories([dataset_dir])
seed_everything(seed=42)

# logging
current_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
log_dir = f"{log_dir_base}/{current_time}"
init_logging_handler(log_dir_base, current_time)

# dump config in logger
with open(f"{log_dir}/config.json", "w") as fp:
    json.dump(train_config, fp)

# saving model path location
model_path_pretrained = os.path.join(
    f"{log_dir}/checkpoint",
    f"{network_type}_pretrained_{dataset_name}_{current_time}.pt",
)

model_path_finetune = os.path.join(
    f"{log_dir}/checkpoint",
    f"{network_type}_finetune_{dataset_name}_{current_time}.pt",
)

In [14]:
# setup default dataset
default_dataset_setup = DefaultDatasetSetupSSL()
default_dataset_setup.setup(dataset_name=dataset_name)
default_dataset_setup.setup_transforms(image_size=image_size)

>>> Multiple Transforms
>>> Multiple Transforms


In [15]:
# trainset
train_dataset = default_dataset_setup.get_dataset(mode="train")
# default_dataset_setup.visualize_dataset(train_dataset)

Model_II dataset already exists
train data: 89104


In [16]:
# split in train and valid set
split_ratio = 0.25  # 0.25
valid_len = int(split_ratio * len(train_dataset))
train_len = len(train_dataset) - valid_len

train_dataset, val_set = random_split(train_dataset, [train_len, valid_len])

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
)

val_loader = DataLoader(
    dataset=val_set, batch_size=batch_size, shuffle=True, num_workers=num_workers
)

# Load test dataset
# testset = default_dataset_setup.get_dataset(mode="val")
# test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=True)

# size check
sample = next(iter(train_loader))
print("num of classes: ", num_classes)
print(sample[0].shape)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")


num of classes:  3
torch.Size([64, 1, 224, 224])


In [17]:
# Create pretrain model
resnet = torchvision.models.resnet50()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = BYOLSingleChannel(backbone, num_ftrs=2048)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
# summary(model, input_size=(1, 1, 224, 224), device="cuda")

########################## Pretraining #############################

# optimizer and loss function for pretrain
optimizer_pretrain = torch.optim.SGD(model.parameters(), lr=0.06)

# criterion
# criterion_pretrain = NegativeCosineSimilarity()s
criterion_pretrain = SymNegCosineSimilarityLoss()

/content/self_supervised/losses/sym_neg_cos_sim_loss.py:35: Warning: SymNegCosineSimiliarityLoss will be deprecated in favor of NegativeCosineSimilarity in the future.
  warnings.warn(


In [ ]:
# pretraining
train_byol(
    epochs=epochs_pretrained,
    model=model,
    device=device,
    train_loader=train_loader,
    criterion=criterion_pretrain,
    optimizer=optimizer_pretrain,
    saved_model_path=model_path_pretrained,
    valid_loader=val_loader,
)

Starting Training


/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gr

Epoch [0/10], Batch [0/1045], Loss: -0.0018986132927238941
Epoch [0/10], Batch [10/1045], Loss: -0.6732980012893677
Epoch [0/10], Batch [20/1045], Loss: -0.762669563293457
Epoch [0/10], Batch [30/1045], Loss: -0.8373775482177734
Epoch [0/10], Batch [40/1045], Loss: -0.8390130996704102
Epoch [0/10], Batch [50/1045], Loss: -0.9137295484542847
Epoch [0/10], Batch [60/1045], Loss: -0.9012939929962158
Epoch [0/10], Batch [70/1045], Loss: -0.9290071725845337
Epoch [0/10], Batch [80/1045], Loss: -0.8955908417701721
Epoch [0/10], Batch [90/1045], Loss: -0.9026325941085815
Epoch [0/10], Batch [100/1045], Loss: -0.9176199436187744
Epoch [0/10], Batch [110/1045], Loss: -0.9116210341453552
Epoch [0/10], Batch [120/1045], Loss: -0.9298646450042725
Epoch [0/10], Batch [130/1045], Loss: -0.9056341648101807
Epoch [0/10], Batch [140/1045], Loss: -0.9072104096412659
Epoch [0/10], Batch [150/1045], Loss: -0.9612534046173096
Epoch [0/10], Batch [160/1045], Loss: -0.93021160364151
Epoch [0/10], Batch [170/

/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gr

Epoch [1/10], Batch [0/1045], Loss: -0.9911845326423645
Epoch [1/10], Batch [10/1045], Loss: -0.9897962808609009
Epoch [1/10], Batch [20/1045], Loss: -0.985688328742981
Epoch [1/10], Batch [30/1045], Loss: -0.9864813089370728
Epoch [1/10], Batch [40/1045], Loss: -0.993876576423645
Epoch [1/10], Batch [50/1045], Loss: -0.9837156534194946
Epoch [1/10], Batch [60/1045], Loss: -0.9895741939544678
Epoch [1/10], Batch [70/1045], Loss: -0.9922720193862915
Epoch [1/10], Batch [80/1045], Loss: -0.9791045784950256
Epoch [1/10], Batch [90/1045], Loss: -0.9841586947441101
Epoch [1/10], Batch [100/1045], Loss: -0.9893655776977539
Epoch [1/10], Batch [110/1045], Loss: -0.9841111898422241
Epoch [1/10], Batch [120/1045], Loss: -0.990739107131958
Epoch [1/10], Batch [130/1045], Loss: -0.9912154078483582
Epoch [1/10], Batch [140/1045], Loss: -0.9903329014778137
Epoch [1/10], Batch [150/1045], Loss: -0.9941809773445129
Epoch [1/10], Batch [160/1045], Loss: -0.9908374547958374
Epoch [1/10], Batch [170/104

/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/transforms.py:1523: UserWarning: The image is already gr

Epoch [2/10], Batch [0/1045], Loss: -0.9941126704216003
Epoch [2/10], Batch [10/1045], Loss: -0.994738757610321
Epoch [2/10], Batch [20/1045], Loss: -0.9937441349029541
Epoch [2/10], Batch [30/1045], Loss: -0.9958057403564453
Epoch [2/10], Batch [40/1045], Loss: -0.9866570234298706
Epoch [2/10], Batch [50/1045], Loss: -0.9929462671279907
Epoch [2/10], Batch [60/1045], Loss: -0.988060712814331
Epoch [2/10], Batch [70/1045], Loss: -0.9909249544143677
Epoch [2/10], Batch [80/1045], Loss: -0.9954149723052979
Epoch [2/10], Batch [90/1045], Loss: -0.9913302659988403
Epoch [2/10], Batch [100/1045], Loss: -0.9847187995910645
Epoch [2/10], Batch [110/1045], Loss: -0.9968182444572449
Epoch [2/10], Batch [120/1045], Loss: -0.9829838871955872
Epoch [2/10], Batch [130/1045], Loss: -0.9877912998199463
Epoch [2/10], Batch [140/1045], Loss: -0.9950253963470459
Epoch [2/10], Batch [150/1045], Loss: -0.9944820404052734
Epoch [2/10], Batch [160/1045], Loss: -0.9834882020950317
Epoch [2/10], Batch [170/10

In [ ]:
!nvidia-smi

# New Section